In [64]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.parser import parse

metadata_file = "output/App_metadata.csv"
rel_notes_folder = "Automobile_extracts/"
NUM_SAMPLE_APPS = 30

In [69]:
##Read metadata for automobile apps collected ( 30 apps from each category). 
df_metadata = pd.read_csv(metadata_file)
df_metadata
df_metadata["category"].unique() #Check categories, we are going to experiment on automotive
# array(['automotive', 'books', 'business', 'education', 'entertainment',
#        'family and relationship', 'finance', 'food', 'graphic and design',
#        'game', 'health and fitness', 'lifestyle', 'news and magazines',
#        'photo and video', 'real estate', 'shopping', 'social media',
#        'sports', 'travel', 'utility and productivity', 'games'],
#       dtype=object)

auto_meta_df = df_metadata[df_metadata["category"]== "automotive"]
auto_meta_df
app_names = auto_meta_df['title'].values

##Read the release notes of the apps

# Import libraries
import glob

# Get CSV files list from a folder
csv_files = glob.glob(rel_notes_folder + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)

#Sort the dataframe according to date decreasing order

# for i in df_list:
#     print(i)
# # print(len(df_list))
#for every df in the list add its respective app_name
big_df = pd.DataFrame() # Create empty dataframe
# app_nm_j = 0 #To iterate categories
for i in df_list:
#     i.insert(loc = 0, column = "app_name", value = app_names[app_nm_j], allow_duplicates=False)
#     app_nm_j = app_nm_j + 1
    # Concatenate all DataFrames
    big_df   = pd.concat([big_df,i], ignore_index=True)
big_df




,App,Ver,Date,Notes
0,Simply Auto: Car Maintenance &,50.1.1.1,"Mar 24, 2022",Bug fix for overlapping text
1,Simply Auto: Car Maintenance &,49.1.1.1,"Mar 10, 2022",-Updated save icon-Increased font size on the ...
2,Simply Auto: Car Maintenance &,48.1.1.1,"Feb 17, 2022",Bugfix for Total Cost calculation
3,Simply Auto: Car Maintenance &,47.1.1.3,"Feb 7, 2022","-Easy access to delete Vehicles, Fill-ups, ser..."
4,Simply Auto: Car Maintenance &,46.1.2.1,"Jan 22, 2022",- Fixed crashes- Fixed service type delete bug
...,...,...,...,...
767,CAR PROBLEMS AND REPAIRS,2.2.1,"Nov 2, 2020",Level improvements!
768,CAR PROBLEMS AND REPAIRS,2.2.0,"Oct 24, 2020",New mechanic and reward system come with this ...
769,CAR PROBLEMS AND REPAIRS,2.1.9,"Oct 15, 2020",New features and performance improvements!
770,CAR PROBLEMS AND REPAIRS,2.1.7,"Oct 8, 2020",Performance improvements!


In [70]:
big_df.describe(include = 'all')

,App,Ver,Date,Notes
count,766,770,768,766
unique,30,640,612,401
top,Android Auto,3.0.0,19 Dec 2022,• Improved Do Not Disturb functionality.\n• Da...
freq,105,5,5,83


In [71]:
big_df.info()
#Drop the rows with null data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 772 entries, 0 to 771
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   App     766 non-null    object
 1   Ver     770 non-null    object
 2   Date    768 non-null    object
 3   Notes   766 non-null    object
dtypes: object(4)
memory usage: 24.2+ KB


In [72]:
#Drop the rows with null data
big_df = big_df.dropna()
big_df.info()
big_df = big_df.reset_index(drop=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 757 entries, 0 to 771
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   App     757 non-null    object
 1   Ver     757 non-null    object
 2   Date    757 non-null    object
 3   Notes   757 non-null    object
dtypes: object(4)
memory usage: 29.6+ KB


In [73]:
UNIQUE_ROWS = big_df.shape[0]
# Convert to the traditional release version
big_df = big_df.astype({'Ver':'string'}) #Convert from float to string      
for i in range(UNIQUE_ROWS):
    dot_count = big_df["Ver"][i].count('.')
#     print(dot_count)
    if dot_count ==0:
        big_df["Ver"][i] = big_df["Ver"][i]+".0.0" #append .0.0
    elif dot_count ==1:
        big_df["Ver"][i] = big_df["Ver"][i]+".0" #append .0
    if dot_count <2:  #Validation to check no non-traditional string
        print(big_df["Ver"][i])
    
#Add a column to find difference in dates
big_df.insert(loc = 4, column = "Time of Rel", value = np.nan, allow_duplicates=False)
for i in range(UNIQUE_ROWS - 1):
#     datetime.strptime(big_df["App"][i])
    parse(big_df["Date"][i])
    if big_df["App"][i] == big_df["App"][i + 1]:
#         datetime.strptime(date_string)
        delta = parse(big_df["Date"][i]) - parse(big_df["Date"][i + 1])
        print(delta.days)
        
        
#Remove the first date in every app as we dont know the difference to its prev release.
# big_df = big_df.dropna()
# big_df.info()
# big_df = big_df.reset_index(drop=True)
# UNIQUE_ROWS = big_df.shape[0]

#Categorize them as major, minor, patch releases
# big_df.insert(loc = 5, column = "Rel type", value = np.nan, allow_duplicates=False)
# for i in range(UNIQUE_ROWS - 1):
#     if big_df["App"][i] == big_df["App"][i + 1]:
#         if diff b/w number before 1st . is > 0:
#             big_df["Rel type"][i] = "Major"
#         elif diff b/w number after 1st before 2nd . is > 0:
#             big_df["Rel type"][i] = "Minor"
#         else
#             big_df["Rel type"][i] = "Patch"
# big_df = big_df.dropna()
# big_df.info()
# big_df = big_df.reset_index(drop=True)
# UNIQUE_ROWS = big_df.shape[0]

#Find length, updatability, updatability for major releases

3.2.0
2.0.0
6.7.0
6.6.0
6.5.0
6.4.0
6.3.0
6.2.0
6.1.0
6.0.0
3.0.0
2.6.0
2.59.0
2.58.0
2.57.0
2.56.0
2.55.0
2.54.0
2.53.0
2.52.0
2.51.0
2.5.0
2.49.0
2.48.0
2.47.0
2.46.0
2.45.0
2.44.0
2.43.0
2.42.0
2.41.0
2.4.0
2.39.0
2.38.0
2.37.0
2.36.0
2.562.0
3.2.0
2.0.0
3.2.0
2.36.0
7.3.0
7.2.0
7.1.0
7.0.0
5.7.0
5.6.0
5.5.0
5.4.0
5.3.0
5.2.0
5.1.0
5.0.0
2.5.0
14
21
10
16
10
18
1
73
38
10
28
58
19
41
29
20
15
12
15
56
22
28
42
61
12
13
26
33
16
10
61
8
12
9
5
1
4
11
1
31
57
63
6
5
11
19
51
27
148
37
72
9
7
18
3
203
5
132
18
46
20
101
143
7
37
236
232
17
54
91
24
5
9
0
13
808
2
61
5
1
640
6
3
1
10
3
1
4
9
4
23
7
6
9
-50
57
6
6
6
2
6
6
3
11
3
22
-4
0
3
0
10
5
10
8
13
8
13
4
19
6
6
9
8
11
16
5
9
6
8
12
17
12
1
2
12
13
62
7
15
16
12
9
19
6
8
10
5
9
5
15
18
-4
12
21
28
26
30
30
43
21
33
35
34
46
4
30
34
40
41
37
23
41
41
28
37
32
34
35
26
32
48
11
18
12
3
14
5
13
3
3
10
6
13
1
44
24
19
11
31
64
11
7
11
35
21
92
39
21
11
10
51
35
3
6
17
4
61
3
3
20
1
17
18
6
21
11
147
32
73
66
52
105
2045
260
319
244
123
